In [ ]:
# Cell 1: Header and Setup
import sys
sys.path.insert(0, '..')

from utils.notebook_utils import display_header, display_toc, check_dependency, conclusion_box, info_box, warning_box
from utils.system_info import display_system_info
from utils.benchmark import Benchmark, BenchmarkResult, ComparisonTable
from utils.charts import setup_style, bar_comparison, throughput_comparison, COLORS

display_header('End-to-End ML Pipeline', 'SynaDB as Unified Data Layer')

In [ ]:
# Cell 2: Table of Contents
sections = [
    ('Introduction', 'introduction'),
    ('Setup', 'setup'),
    ('Data Ingestion', 'data-ingestion'),
    ('Feature Engineering', 'feature-engineering'),
    ('Model Training', 'model-training'),
    ('Experiment Tracking', 'experiment-tracking'),
    ('Model Registry', 'model-registry'),
    ('Inference', 'inference'),
    ('Unified Data Layer', 'unified-data'),
    ('Results Summary', 'results'),
    ('Conclusions', 'conclusions'),
]
display_toc(sections)

## 📌 Introduction <a id="introduction"></a>

This notebook demonstrates **SynaDB as a unified data layer** for the complete ML lifecycle.

### The ML Pipeline Challenge

Traditional ML pipelines require multiple tools:

```
┌─────────────┐   ┌─────────────┐   ┌─────────────┐
│ Data Lake   │ → │ Feature     │ → │ Training    │
│ (S3/HDFS)   │   │ Store       │   │ (PyTorch)   │
└─────────────┘   │ (Feast)     │   └─────────────┘
                  └─────────────┘          ↓
┌─────────────┐   ┌─────────────┐   ┌─────────────┐
│ Inference   │ ← │ Model       │ ← │ Experiment  │
│ Service     │   │ Registry    │   │ Tracking    │
└─────────────┘   │ (MLflow)    │   │ (W&B)       │
                  └─────────────┘   └─────────────┘
```

### SynaDB Unified Approach

```
┌─────────────────────────────────────────────────┐
│                    SynaDB                        │
├─────────────────────────────────────────────────┤
│ Raw Data │ Features │ Experiments │ Models      │
│          │          │             │             │
│ Vectors  │ Tensors  │ Metrics     │ Artifacts   │
└─────────────────────────────────────────────────┘
```

### What We'll Build

A complete ML pipeline for image classification:

1. **Data Ingestion** - Load and store training data
2. **Feature Engineering** - Extract and store features
3. **Model Training** - Train with SynaDB DataLoader
4. **Experiment Tracking** - Log metrics and parameters
5. **Model Registry** - Version and store models
6. **Inference** - Load model and make predictions

In [ ]:
# Cell 4: System Info
display_system_info()

## 🔧 Setup <a id="setup"></a>

Let's set up our environment for the end-to-end pipeline.

In [ ]:
# Cell 6: Setup
import numpy as np
import time
import os
import tempfile
import hashlib
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple
import matplotlib.pyplot as plt

# Check dependencies
HAS_SYNADB = check_dependency('synadb', 'pip install synadb')
HAS_TORCH = check_dependency('torch', 'pip install torch', required=False)
HAS_SKLEARN = check_dependency('sklearn', 'pip install scikit-learn', required=False)

# Apply consistent styling
setup_style()

# Create temp directory
temp_dir = tempfile.mkdtemp(prefix='synadb_pipeline_')
print(f'Using temp directory: {temp_dir}')

# Pipeline configuration
NUM_SAMPLES = 1000
NUM_FEATURES = 64
NUM_CLASSES = 10
TRAIN_SPLIT = 0.8

print(f"\n✓ Setup complete")
print(f"  Samples: {NUM_SAMPLES:,}")
print(f"  Features: {NUM_FEATURES}")
print(f"  Classes: {NUM_CLASSES}")
print(f"  Train/Test split: {TRAIN_SPLIT}/{1-TRAIN_SPLIT}")

## 📥 Data Ingestion <a id="data-ingestion"></a>

The first stage loads raw data into SynaDB.

### Data Schema

| Key Pattern | Type | Description |
|-------------|------|-------------|
| `raw/sample/{id}/features` | Vector | Raw feature vector |
| `raw/sample/{id}/label` | Int | Class label |
| `raw/metadata/num_samples` | Int | Total samples |
| `raw/metadata/num_features` | Int | Feature dimensions |

In [ ]:
# Cell 8: Generate and Ingest Data
np.random.seed(42)

# Generate synthetic classification data
def generate_synthetic_data(n_samples, n_features, n_classes):
    """Generate synthetic classification data with cluster structure."""
    X = []
    y = []
    
    samples_per_class = n_samples // n_classes
    
    for class_id in range(n_classes):
        # Each class has a different center
        center = np.random.randn(n_features) * 2
        
        for _ in range(samples_per_class):
            # Add noise around the center
            sample = center + np.random.randn(n_features) * 0.5
            X.append(sample.astype(np.float32))
            y.append(class_id)
    
    return np.array(X), np.array(y)

X_raw, y_raw = generate_synthetic_data(NUM_SAMPLES, NUM_FEATURES, NUM_CLASSES)
print(f"Generated synthetic data:")
print(f"  X shape: {X_raw.shape}")
print(f"  y shape: {y_raw.shape}")
print(f"  Class distribution: {np.bincount(y_raw)}")

In [ ]:
# Cell 9: Store Raw Data in SynaDB
if HAS_SYNADB:
    from synadb import SynaDB, VectorStore
    
    db_path = os.path.join(temp_dir, 'pipeline.db')
    db = SynaDB(db_path)
    
    print("Ingesting raw data into SynaDB...\n")
    
    start = time.perf_counter()
    
    # Store metadata
    db.put_int('raw/metadata/num_samples', NUM_SAMPLES)
    db.put_int('raw/metadata/num_features', NUM_FEATURES)
    db.put_int('raw/metadata/num_classes', NUM_CLASSES)
    db.put_text('raw/metadata/created_at', datetime.now().isoformat())
    
    # Store samples
    for i, (features, label) in enumerate(zip(X_raw, y_raw)):
        # Store features as floats (one per dimension for simplicity)
        for j, val in enumerate(features):
            db.put_float(f'raw/sample/{i:04d}/feature/{j:02d}', float(val))
        db.put_int(f'raw/sample/{i:04d}/label', int(label))
    
    ingestion_time = (time.perf_counter() - start) * 1000
    
    print(f"✓ Ingested {NUM_SAMPLES:,} samples in {ingestion_time:.1f}ms")
    print(f"  Throughput: {NUM_SAMPLES / (ingestion_time / 1000):.0f} samples/sec")
    print(f"  File size: {os.path.getsize(db_path) / 1024:.1f} KB")
else:
    warning_box("SynaDB not installed - skipping data ingestion")
    db = None

## 🔧 Feature Engineering <a id="feature-engineering"></a>

Transform raw data into ML-ready features.

### Feature Transformations

| Transformation | Description |
|----------------|-------------|
| Normalization | Scale to zero mean, unit variance |
| PCA | Dimensionality reduction |
| Feature Selection | Keep top-k features |

In [ ]:
# Cell 11: Feature Engineering
if HAS_SYNADB and db:
    print("Performing feature engineering...\n")
    
    # Load raw data from SynaDB
    start = time.perf_counter()
    
    X_loaded = []
    y_loaded = []
    
    for i in range(NUM_SAMPLES):
        features = []
        for j in range(NUM_FEATURES):
            val = db.get_float(f'raw/sample/{i:04d}/feature/{j:02d}')
            features.append(val if val is not None else 0.0)
        X_loaded.append(features)
        y_loaded.append(db.get_int(f'raw/sample/{i:04d}/label'))
    
    X_loaded = np.array(X_loaded, dtype=np.float32)
    y_loaded = np.array(y_loaded)
    
    load_time = (time.perf_counter() - start) * 1000
    print(f"Loaded raw data in {load_time:.1f}ms")
    
    # Normalize features
    mean = X_loaded.mean(axis=0)
    std = X_loaded.std(axis=0) + 1e-8
    X_normalized = (X_loaded - mean) / std
    
    # Store normalization parameters
    for j in range(NUM_FEATURES):
        db.put_float(f'features/norm/mean/{j:02d}', float(mean[j]))
        db.put_float(f'features/norm/std/{j:02d}', float(std[j]))
    
    # Store normalized features
    start = time.perf_counter()
    
    for i, (features, label) in enumerate(zip(X_normalized, y_loaded)):
        for j, val in enumerate(features):
            db.put_float(f'features/sample/{i:04d}/normalized/{j:02d}', float(val))
    
    store_time = (time.perf_counter() - start) * 1000
    
    print(f"\n✓ Feature engineering complete")
    print(f"  Normalization: mean={mean.mean():.4f}, std={std.mean():.4f}")
    print(f"  Stored normalized features in {store_time:.1f}ms")
else:
    print("Database not available for feature engineering")

In [ ]:
# Cell 12: Train/Test Split
if HAS_SYNADB and db:
    print("Creating train/test split...\n")
    
    # Shuffle indices
    indices = np.random.permutation(NUM_SAMPLES)
    split_idx = int(NUM_SAMPLES * TRAIN_SPLIT)
    
    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]
    
    # Store split information
    db.put_int('split/train_size', len(train_indices))
    db.put_int('split/test_size', len(test_indices))
    
    for i, idx in enumerate(train_indices):
        db.put_int(f'split/train/{i:04d}', int(idx))
    
    for i, idx in enumerate(test_indices):
        db.put_int(f'split/test/{i:04d}', int(idx))
    
    print(f"✓ Split created")
    print(f"  Train samples: {len(train_indices):,}")
    print(f"  Test samples: {len(test_indices):,}")
    
    # Prepare numpy arrays for training
    X_train = X_normalized[train_indices]
    y_train = y_loaded[train_indices]
    X_test = X_normalized[test_indices]
    y_test = y_loaded[test_indices]
    
    print(f"  X_train shape: {X_train.shape}")
    print(f"  X_test shape: {X_test.shape}")
else:
    print("Database not available for split")

## 🎯 Model Training <a id="model-training"></a>

Train a classifier using data from SynaDB.

### Training Options

| Framework | Integration |
|-----------|-------------|
| PyTorch | SynaDataset + DataLoader |
| scikit-learn | NumPy arrays from SynaDB |
| TensorFlow | tf.data.Dataset |

In [ ]:
# Cell 14: Simple Classifier Training
training_metrics = []

if HAS_SYNADB and db:
    print("Training classifier...\n")
    
    # Simple logistic regression-style classifier
    class SimpleClassifier:
        def __init__(self, n_features, n_classes, lr=0.01):
            self.W = np.random.randn(n_features, n_classes) * 0.01
            self.b = np.zeros(n_classes)
            self.lr = lr
        
        def softmax(self, x):
            exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
            return exp_x / exp_x.sum(axis=1, keepdims=True)
        
        def forward(self, X):
            return self.softmax(X @ self.W + self.b)
        
        def loss(self, X, y):
            probs = self.forward(X)
            n = len(y)
            return -np.log(probs[np.arange(n), y] + 1e-8).mean()
        
        def accuracy(self, X, y):
            preds = self.forward(X).argmax(axis=1)
            return (preds == y).mean()
        
        def train_step(self, X, y):
            n = len(y)
            probs = self.forward(X)
            
            # Gradient
            grad = probs.copy()
            grad[np.arange(n), y] -= 1
            grad /= n
            
            # Update
            self.W -= self.lr * (X.T @ grad)
            self.b -= self.lr * grad.sum(axis=0)
    
    # Train
    model = SimpleClassifier(NUM_FEATURES, NUM_CLASSES, lr=0.1)
    n_epochs = 50
    batch_size = 32
    
    start = time.perf_counter()
    
    for epoch in range(n_epochs):
        # Shuffle training data
        perm = np.random.permutation(len(X_train))
        X_shuffled = X_train[perm]
        y_shuffled = y_train[perm]
        
        # Mini-batch training
        for i in range(0, len(X_train), batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]
            model.train_step(X_batch, y_batch)
        
        # Log metrics
        train_loss = model.loss(X_train, y_train)
        train_acc = model.accuracy(X_train, y_train)
        test_acc = model.accuracy(X_test, y_test)
        
        training_metrics.append({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_acc': test_acc
        })
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d}: loss={train_loss:.4f}, train_acc={train_acc:.4f}, test_acc={test_acc:.4f}")
    
    training_time = (time.perf_counter() - start) * 1000
    
    final_acc = model.accuracy(X_test, y_test)
    print(f"\n✓ Training complete in {training_time:.1f}ms")
    print(f"  Final test accuracy: {final_acc:.4f}")
else:
    print("Database not available for training")

In [ ]:
# Cell 15: Visualize Training
if training_metrics:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    epochs = [m['epoch'] for m in training_metrics]
    
    # Loss curve
    axes[0].plot(epochs, [m['train_loss'] for m in training_metrics], 
                 color=COLORS['synadb'], linewidth=2, label='Train Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    # Accuracy curves
    axes[1].plot(epochs, [m['train_acc'] for m in training_metrics], 
                 color=COLORS['synadb'], linewidth=2, label='Train Acc')
    axes[1].plot(epochs, [m['test_acc'] for m in training_metrics], 
                 color=COLORS['competitor'], linewidth=2, label='Test Acc')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training & Test Accuracy', fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No training metrics to visualize")

## 📊 Experiment Tracking <a id="experiment-tracking"></a>

Log all training metrics and parameters to SynaDB.

### What We Track

| Category | Items |
|----------|-------|
| **Parameters** | learning_rate, batch_size, epochs |
| **Metrics** | loss, train_acc, test_acc per epoch |
| **Artifacts** | Model weights, normalization params |

In [ ]:
# Cell 17: Log Experiment
if HAS_SYNADB and db and training_metrics:
    print("Logging experiment to SynaDB...\n")
    
    import uuid
    run_id = str(uuid.uuid4())[:8]
    
    # Log parameters
    params = {
        'learning_rate': 0.1,
        'batch_size': 32,
        'epochs': 50,
        'n_features': NUM_FEATURES,
        'n_classes': NUM_CLASSES,
        'train_size': len(X_train),
        'test_size': len(X_test),
    }
    
    for key, value in params.items():
        if isinstance(value, float):
            db.put_float(f'experiments/{run_id}/params/{key}', value)
        else:
            db.put_int(f'experiments/{run_id}/params/{key}', value)
    
    # Log metrics
    for m in training_metrics:
        epoch = m['epoch']
        db.put_float(f'experiments/{run_id}/metrics/train_loss/{epoch:03d}', m['train_loss'])
        db.put_float(f'experiments/{run_id}/metrics/train_acc/{epoch:03d}', m['train_acc'])
        db.put_float(f'experiments/{run_id}/metrics/test_acc/{epoch:03d}', m['test_acc'])
    
    # Log final metrics
    final = training_metrics[-1]
    db.put_float(f'experiments/{run_id}/final/train_loss', final['train_loss'])
    db.put_float(f'experiments/{run_id}/final/train_acc', final['train_acc'])
    db.put_float(f'experiments/{run_id}/final/test_acc', final['test_acc'])
    
    # Log metadata
    db.put_text(f'experiments/{run_id}/metadata/created_at', datetime.now().isoformat())
    db.put_text(f'experiments/{run_id}/metadata/status', 'completed')
    
    print(f"✓ Experiment logged")
    print(f"  Run ID: {run_id}")
    print(f"  Parameters: {len(params)}")
    print(f"  Metric series: 3 (train_loss, train_acc, test_acc)")
    print(f"  Epochs logged: {len(training_metrics)}")
else:
    run_id = None
    print("Database not available for experiment logging")

## 📦 Model Registry <a id="model-registry"></a>

Version and store trained models with checksums.

### Model Versioning

| Field | Description |
|-------|-------------|
| **version** | Auto-incrementing version number |
| **checksum** | SHA-256 hash for integrity |
| **stage** | Development → Staging → Production |
| **metrics** | Associated performance metrics |

In [ ]:
# Cell 19: Save Model to Registry
if HAS_SYNADB and db and 'model' in dir():
    print("Saving model to registry...\n")
    
    model_name = 'simple_classifier'
    
    # Get current version
    version_key = f'models/{model_name}/latest_version'
    current_version = db.get_int(version_key)
    new_version = (current_version or 0) + 1
    
    # Serialize model weights
    model_bytes = model.W.tobytes() + model.b.tobytes()
    checksum = hashlib.sha256(model_bytes).hexdigest()
    
    # Store model
    version_prefix = f'models/{model_name}/v{new_version:03d}'
    
    # Store weights as individual floats (for demo)
    for i in range(model.W.shape[0]):
        for j in range(model.W.shape[1]):
            db.put_float(f'{version_prefix}/weights/W/{i:02d}/{j:02d}', float(model.W[i, j]))
    
    for j in range(len(model.b)):
        db.put_float(f'{version_prefix}/weights/b/{j:02d}', float(model.b[j]))
    
    # Store metadata
    db.put_text(f'{version_prefix}/checksum', checksum)
    db.put_text(f'{version_prefix}/stage', 'development')
    db.put_text(f'{version_prefix}/created_at', datetime.now().isoformat())
    db.put_float(f'{version_prefix}/metrics/test_acc', final['test_acc'])
    
    if run_id:
        db.put_text(f'{version_prefix}/experiment_id', run_id)
    
    # Update latest version
    db.put_int(version_key, new_version)
    
    print(f"✓ Model saved to registry")
    print(f"  Name: {model_name}")
    print(f"  Version: {new_version}")
    print(f"  Checksum: {checksum[:16]}...")
    print(f"  Stage: development")
    print(f"  Test accuracy: {final['test_acc']:.4f}")
else:
    print("Model not available for registry")

In [ ]:
# Cell 20: Promote Model
if HAS_SYNADB and db:
    print("Promoting model to production...\n")
    
    # Update stage
    db.put_text(f'{version_prefix}/stage', 'production')
    
    # Set as production model
    db.put_int(f'models/{model_name}/production_version', new_version)
    
    print(f"✓ Model promoted")
    print(f"  {model_name} v{new_version} → production")
else:
    print("Database not available for promotion")

## 🔮 Inference <a id="inference"></a>

Load the production model and make predictions.

In [ ]:
# Cell 22: Load and Inference
if HAS_SYNADB and db:
    print("Loading production model for inference...\n")
    
    # Get production version
    prod_version = db.get_int(f'models/{model_name}/production_version')
    version_prefix = f'models/{model_name}/v{prod_version:03d}'
    
    # Load weights
    W_loaded = np.zeros((NUM_FEATURES, NUM_CLASSES))
    b_loaded = np.zeros(NUM_CLASSES)
    
    for i in range(NUM_FEATURES):
        for j in range(NUM_CLASSES):
            W_loaded[i, j] = db.get_float(f'{version_prefix}/weights/W/{i:02d}/{j:02d}')
    
    for j in range(NUM_CLASSES):
        b_loaded[j] = db.get_float(f'{version_prefix}/weights/b/{j:02d}')
    
    # Verify checksum
    loaded_bytes = W_loaded.tobytes() + b_loaded.tobytes()
    loaded_checksum = hashlib.sha256(loaded_bytes).hexdigest()
    stored_checksum = db.get_text(f'{version_prefix}/checksum')
    
    print(f"Model loaded: {model_name} v{prod_version}")
    print(f"Checksum verification: {'✓ PASSED' if loaded_checksum == stored_checksum else '✗ FAILED'}")
    
    # Create inference model
    inference_model = SimpleClassifier(NUM_FEATURES, NUM_CLASSES)
    inference_model.W = W_loaded
    inference_model.b = b_loaded
    
    # Run inference on test set
    start = time.perf_counter()
    predictions = inference_model.forward(X_test).argmax(axis=1)
    inference_time = (time.perf_counter() - start) * 1000
    
    accuracy = (predictions == y_test).mean()
    
    print(f"\n✓ Inference complete")
    print(f"  Samples: {len(X_test):,}")
    print(f"  Time: {inference_time:.2f}ms")
    print(f"  Throughput: {len(X_test) / (inference_time / 1000):,.0f} samples/sec")
    print(f"  Accuracy: {accuracy:.4f}")
else:
    print("Database not available for inference")

## 🗄️ Unified Data Layer <a id="unified-data"></a>

Let's examine what's stored in our single SynaDB file.

In [ ]:
# Cell 24: Data Layer Summary
if HAS_SYNADB and db:
    from IPython.display import display, Markdown
    
    all_keys = db.keys()
    
    # Categorize keys
    categories = {
        'raw/': 'Raw Data',
        'features/': 'Features',
        'split/': 'Train/Test Split',
        'experiments/': 'Experiments',
        'models/': 'Model Registry',
    }
    
    counts = {cat: 0 for cat in categories.values()}
    for key in all_keys:
        for prefix, cat in categories.items():
            if key.startswith(prefix):
                counts[cat] += 1
                break
    
    summary = f"""
### Unified Data Layer Contents

| Category | Keys | Description |
|----------|------|-------------|
| **Raw Data** | {counts['Raw Data']:,} | Original samples and metadata |
| **Features** | {counts['Features']:,} | Normalized features and params |
| **Train/Test Split** | {counts['Train/Test Split']:,} | Split indices |
| **Experiments** | {counts['Experiments']:,} | Params, metrics, artifacts |
| **Model Registry** | {counts['Model Registry']:,} | Versioned models |
| **Total** | {len(all_keys):,} | All data in one file |

### File Size

- **Database**: {os.path.getsize(db_path) / 1024:.1f} KB

### Benefits of Unified Storage

1. **Single file** - Easy backup, versioning, sharing
2. **No network** - All data local, zero latency
3. **Atomic** - Consistent state across all components
4. **Portable** - Works on any platform
5. **Reproducible** - Complete pipeline state in one place
"""
    
    display(Markdown(summary))
else:
    print("Database not available for summary")

## 📊 Results Summary <a id="results"></a>

In [ ]:
# Cell 26: Results Summary
from IPython.display import display, Markdown

summary = """
### Pipeline Performance Summary

| Stage | Status | Notes |
|-------|--------|-------|
| **Data Ingestion** | ✓ | Raw data stored |
| **Feature Engineering** | ✓ | Normalized features |
| **Model Training** | ✓ | Classifier trained |
| **Experiment Tracking** | ✓ | Metrics logged |
| **Model Registry** | ✓ | Model versioned |
| **Inference** | ✓ | Predictions made |

### SynaDB vs Traditional Stack

| Aspect | Traditional | SynaDB |
|--------|-------------|--------|
| **Components** | 5-6 tools | 1 database |
| **Setup** | Hours | Minutes |
| **Network** | Required | None |
| **Storage** | Distributed | Single file |
| **Portability** | Complex | Copy file |

### When to Use SynaDB Pipeline

✅ **Good fit:**
- Individual ML practitioners
- Prototyping and experimentation
- Edge/embedded ML
- Offline development
- Teaching and tutorials

⚠️ **Consider alternatives:**
- Large-scale production systems
- Multi-team collaboration
- Distributed training
"""

display(Markdown(summary))

## 🎯 Conclusions <a id="conclusions"></a>

In [ ]:
# Cell 28: Conclusions
conclusion_box(
    title="Key Takeaways",
    points=[
        "SynaDB serves as a unified data layer for the complete ML lifecycle",
        "All pipeline stages share a single database file",
        "No external services or network required",
        "Model versioning with checksum verification ensures integrity",
        "Experiment tracking captures full training history",
        "Single-file storage simplifies reproducibility and sharing",
    ],
    summary="SynaDB provides a simple, unified approach to ML data management for individual practitioners and small teams."
)

In [ ]:
# Cell 29: Cleanup
import shutil

print("Cleaning up temporary files...")
try:
    if HAS_SYNADB and db:
        db.close()
    shutil.rmtree(temp_dir)
    print(f"✓ Removed temp directory: {temp_dir}")
except Exception as e:
    print(f"⚠️ Could not remove temp directory: {e}")

print("\n✓ Notebook complete!")